# CNN Pipeline v3

## Environment settings

### Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from PIL import Image, ImageFilter, ImageEnhance
from numpy import asarray
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output
import math

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Random seed

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Data Loader

In [ ]:
# google drive plug-in
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# directory
%cd /gdrive/MyDrive/notebooks/an2dl_homeworks/training_validation_testing/mirko

In [5]:
# paths
root_path = '/kaggle/input/leaves/leaves/'#'../../training_validation_testing/mirko/'
training_dir = os.path.join(root_path, 'training')
validation_dir = os.path.join(root_path, 'validation')
testing_dir = os.path.join(root_path, 'testing')

## Data Pre-Processing

### Model statistics

#### Set pre-computed statistics

In [ ]:
# MEAN CENTER SHIFT
z_score_mean = 0

# STD NORMALIZATION
z_score_std = 0

#MEAN :
#[[[21.787233, 21.787233, 21.787233]]]
#STD :
#[[[49.0662,   64.137405, 40.04247 ]]]

#### Compute new statistics

In [9]:
# TRAINING ---------------------------------------------------------------------

# constructor
train_data_gen = ImageDataGenerator(
    # data augmentation
    rotation_range=40,
    height_shift_range=50,
    width_shift_range=50,
    zoom_range=0.3,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5,1.5],
    fill_mode='reflect',

    # z-score
    featurewise_center=False, # mean center shift
    featurewise_std_normalization=False # std normalization
)

# generator
train_gen = train_data_gen.flow_from_directory(
    directory= root_path + 'training',
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

# callable generator method
def CallableGenerator():
  # yielding batch's samples at run time when called from the generator
  for (x,y) in train_gen:
    yield (x,y)

# TENSOR-BATCHED DATASET (from generator)
batched_dataset = tf.data.Dataset.from_generator(
    # above method for run time
    CallableGenerator,

    # tensor fields types
    output_types=(tf.float32, tf.float32)
) 

# TENSOR-UNBATCHED DATASET (from batched-dataset)
mean = tf.Variable(
    # RGB dimension        
    np.zeros((3), dtype=np.float32)
)
mean_2 = tf.Variable(
    # RGB dimension        
    np.zeros((3), dtype=np.float32)
)
std = tf.Variable(
    # RGB dimension        
    np.zeros((3), dtype=np.float32)
)

# visualization variables
i = 0 # counter
tot_epoch = math.ceil(train_gen.samples / 32) # total epoch

# ONE EPOCH LOADING
for (x, _) in iter(batched_dataset):
  # update visualization
  clear_output(wait=True)
  i += 1
  print("Epoch (1/1) - Batch : " + str(i) + "/" + str(tot_epoch))

  # incremental mean
  mean.assign_add(
      tf.math.divide(
        tf.math.reduce_sum(x, axis=[0, 1, 2]),
        256 * 256 * train_gen.samples
      )
  )

  # incremental mean squared
  mean_2.assign_add(
      tf.math.divide(
        tf.math.reduce_sum(tf.math.square(x), axis=[0, 1, 2]),
        256 * 256 * train_gen.samples
      )
  )
    
  # end condition, otherwise unlimited
  if i > tot_epoch:
      break

# standard deviation
std = tf.math.sqrt(
    tf.math.subtract(mean_2, mean)
)

# reshape
mean = tf.reshape(mean, (1, 1, 3))
std = tf.reshape(std, (1, 1, 3))

# visualization
print('training set mean :', mean)
print('training set std :', std)

In [ ]:
# Z-SCORE activation
train_data_gen.featurewise_center = True
train_data_gen.featurewise_std_normalization = True

# statistics fitting computation on unbatched_dataset
train_data_gen.fit(unbatched_dataset)

# new statistics just computed
z_score_mean = train_data_gen.mean
z_score_std = train_data_gen.std

# visualization
print('MEAN :')
print(z_score_mean)
print('STD :')
print(z_score_std)

### Generators

#### Training set

##### Pre-Computed statistics

In [ ]:
# TRAINING ---------------------------------------------------------------------

# constructor
train_data_gen = ImageDataGenerator(
    # z-score
    featurewise_center=True, # mean center shift
    featurewise_std_normalization=True # std normalization
)

# training set statistics
train_data_gen.mean = z_score_mean
train_data_gen.std = z_score_std

# generator
train_gen = train_data_gen.flow_from_directory(
    directory=training_dir,
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

##### Use new statistics

In [ ]:
# generator
train_gen = train_data_gen.flow_from_directory(
    directory=training_dir,
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

#### Validation set

In [ ]:
# VALIDATION -------------------------------------------------------------------

# constructor
valid_data_gen = ImageDataGenerator(
    # z-score
    featurewise_center=True, # mean center shift
    featurewise_std_normalization=True # std normalization
)

# training set statistics
valid_data_gen.mean = z_score_mean
valid_data_gen.std = z_score_std

# generator
valid_gen = valid_data_gen.flow_from_directory(
    directory=validation_dir,
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

#### Testing set

In [ ]:
# TESTING ----------------------------------------------------------------------

# constructor
test_data_gen = ImageDataGenerator(
    # z-score
    featurewise_center=True, # mean center shift
    featurewise_std_normalization=True # std normalization
)

# training set statistics
test_data_gen.mean = z_score_mean
test_data_gen.std = z_score_std

# generator
test_gen = test_data_gen.flow_from_directory(
    directory=testing_dir,
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=8,
    shuffle=True,
    seed=seed
)

In [ ]:
# cross validation
train_valid_data_gen = ImageDataGenerator(preprocessing_function=leaf_preprocess,
                                          rotation_range=30,
                                          height_shift_range=50,
                                          width_shift_range=50,
                                          zoom_range=0.3,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='reflect', #)
                                          rescale=1/255.)

train_valid_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               #color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed)

## Model Design

In [ ]:
# labels
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [ ]:
# Model hyperparameters
input_shape = (256, 256, 3)
epochs = 100
batch_size = 64
n_classes = len(labels)
weight_decay = 1e-3
model_name = "CNN_bw_sharp_gap"

In [ ]:
# Model skeleton
# > (Conv2D + BN + ReLu + MaxPoolAvg) x 3 +
# + (Conv2D + L2 + BN + ReLu + MaxPoolAvg) x 1 + 
# + (Conv2D + BN + ReLu + GlobalPoolAvg) x 1 +
# + SoftMax x 1
def mirknet(input_shape):
    # (0) input
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # (1) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    batch_norm1 = tfkl.BatchNormalization()(conv1)
    relu1 = tf.keras.layers.ReLU()(batch_norm1)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu1)

    # (2) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    batch_norm2 = tfkl.BatchNormalization()(conv2)
    relu2 = tf.keras.layers.ReLU()(batch_norm2)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu2)

    # (3) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    batch_norm3 = tfkl.BatchNormalization()(conv3)
    relu3 = tf.keras.layers.ReLU()(batch_norm3)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu3)

    # (4) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool3)
    batch_norm4 = tfkl.BatchNormalization()(conv4)
    relu4 = tf.keras.layers.ReLU()(batch_norm4)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu4)

    # (5) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    batch_norm5 = tfkl.BatchNormalization()(conv5)
    relu5 = tf.keras.layers.ReLU()(batch_norm5)
    pool5 = tfkl.MaxPooling2D(
        pool_size = (4, 4)
    )(relu5)

    # (6) Output
    flattening_layer = tfkl.Flatten(name='Flatten')(pool5)

    flattening_layer = tfkl.Dropout(0.5, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512,
                                  name='Classifier_1',
                                  kernel_initializer=tfk.initializers.GlorotUniform(seed),
                                  activation='relu')(flattening_layer)

    flattening_layer = tfkl.Dropout(0.5, seed=seed)(classifier_layer)
    classifier_layer = tfkl.Dense(units=256,
                                  name='Classifier_2',
                                  kernel_initializer=tfk.initializers.GlorotUniform(seed),
                                  activation='relu')(flattening_layer)

    output_layer = tfkl.Dense(units=n_classes, activation='softmax', 
                              kernel_initializer=tfk.initializers.GlorotUniform(seed), 
                              name='Output')(classifier_layer)'''

    # Model connection
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Model Compilation
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
# Model skeleton
# > (Conv2D + BN + ReLu + MaxPoolAvg) x 4 +
# + (Conv2D + BN + ReLu + GlobalAvgPool) x 1 +
# + SoftMax x 1
def mirknet(input_shape):
    # (0) input
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # (1) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    batch_norm1 = tfkl.BatchNormalization()(conv1)
    relu1 = tf.keras.layers.ReLU()(batch_norm1)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu1)

    # (2) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    batch_norm2 = tfkl.BatchNormalization()(conv2)
    relu2 = tf.keras.layers.ReLU()(batch_norm2)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu2)

    # (3) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    batch_norm3 = tfkl.BatchNormalization()(conv3)
    relu3 = tf.keras.layers.ReLU()(batch_norm3)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu3)

    # (4) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool3)
    batch_norm4 = tfkl.BatchNormalization()(conv4)
    relu4 = tf.keras.layers.ReLU()(batch_norm4)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu4)

    # (5) Conv + L2 + BN + ReLu + MaxPoolAvg
    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    batch_norm5 = tfkl.BatchNormalization()(conv5)
    relu5 = tf.keras.layers.ReLU()(batch_norm5)
    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu5)
    gap = tfkl.GlobalAveragePooling2D()(pool5)

    # (6) Output

    output_layer = tfkl.Dense(units=n_classes, activation='softmax', 
                              kernel_initializer=tfk.initializers.GlorotUniform(seed), 
                              name='Output')(gap)

    # Model connection
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Model Compilation
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
# Build model
model = mirknet(input_shape)
model.summary()

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join(root_path+'/new_model_gap')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [ ]:
# Create folders and callbacks and fit
callbacks = create_folders_and_callbacks(model_name='CNN_full')

# Train the model
history = model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks=[callbacks]
).history

In [ ]:
# Save best epoch model
model.save("models/CNN_full_best_" + str(datetime.now().strftime('%b%d_%H-%M-%S')))

## Cross-Validation

In [ ]:
from sklearn.model_selection import KFold

num_folds = 10

histories = []
scores = []

kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(X_train_val, y_train_val)):

  print("Starting training on fold num: {}".format(fold_idx+1))

  model = mirknet(input_shape)

  history = model.fit(
    x = X_train_val.iloc[train_idx],
    y = y_train_val.iloc[train_idx],
    validation_data=(X_train_val.iloc[valid_idx], y_train_val.iloc[valid_idx]),
    batch_size = batch_size,
    epochs = 100,
    callbacks=[early_stopping]
  ).history

  score = model.evaluate(X_train_val.iloc[valid_idx], y_train_val.iloc[valid_idx])
  scores.append(score[1])

  histories.append(history)

## Testing

In [ ]:
mean_accuracy = 0
n = 0

y_pred = np.array([])
y_test = np.array([])

for i, (sample, target) in enumerate(test_gen):
  y_pred = np.append(y_pred, np.argmax(model.predict(sample)))
  mean_accuracy += 1 if np.argmax(model.predict(sample)) == np.argmax(target) else 0
  n = i
mean_accuracy /= n

print("mean accuracy (test set) : " + str(mean_accuracy))

In [ ]:
prova = np.array([])
prova2 = np.array([1,2,3,4,5,6,7])

for num in prova2:
  prova = np.append(prova, num)
  print(prova)

prova.item(2)

In [ ]:
y_pred = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])
y_test = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2])
labels = ["Cats", "Dogs", "Horses"]

cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()